In [1]:
import math
import numpy as np
import plotly.graph_objects as go

# =============================================================================
# ASTEROID CLASS DEFINITION (2D Position & Velocity)
# =============================================================================
class Asteroid:
    def __init__(self, asteroid_id, x, y, radius, vx, vy):
        self.id = asteroid_id
        self.x = x
        self.y = y
        self.radius = radius
        self.vx = vx
        self.vy = vy

# =============================================================================
# INPUT PARSING FUNCTION
# =============================================================================
def parse_input(file_name):
    asteroids = []
    auto_id = 1
    with open(file_name, "r") as f:
        line_num = 0
        for line in f:
            line_num += 1
            stripped_line = line.strip()
            if not stripped_line:
                continue
            parts = stripped_line.split()
            if len(parts) == 6:
                try:
                    asteroid_id, x, y, radius, vx, vy = map(float, parts)
                    asteroid_id = int(asteroid_id)
                except ValueError:
                    print(f"Warning: Non-numeric values in line {line_num}: '{stripped_line}'. Skipping.")
                    continue
            elif len(parts) == 5:
                try:
                    x, y, radius, vx, vy = map(float, parts)
                except ValueError:
                    print(f"Warning: Non-numeric values in line {line_num}: '{stripped_line}'. Skipping.")
                    continue
                asteroid_id = auto_id
                auto_id += 1
            else:
                print(f"Warning: Incorrect field count in line {line_num}: '{stripped_line}'. Skipping.")
                continue
            asteroids.append(Asteroid(asteroid_id, x, y, radius, vx, vy))
    if not asteroids:
        raise ValueError("No valid asteroid data found in the input file.")
    return asteroids

# =============================================================================
# COLLISION DETECTION FUNCTION
# =============================================================================
def detect_collisions(asteroids, time_step, max_time):
    collision_dict = {}
    num_steps = int(max_time / time_step)
    
    for step in range(num_steps + 1):
        t = step * time_step
        for i in range(len(asteroids)):
            for j in range(i + 1, len(asteroids)):
                key = (asteroids[i].id, asteroids[j].id)
                if key in collision_dict:
                    continue
                a1 = asteroids[i]
                a2 = asteroids[j]
                x1, y1 = a1.x + a1.vx * t, a1.y + a1.vy * t
                x2, y2 = a2.x + a2.vx * t, a2.y + a2.vy * t
                distance = math.hypot(x2 - x1, y2 - y1)
                if distance < (a1.radius + a2.radius):
                    collision_dict[key] = t
    collisions = [(t, id1, id2) for (id1, id2), t in collision_dict.items()]
    collisions.sort(key=lambda x: (x[0], x[1], x[2]))
    return collisions

# =============================================================================
# OUTPUT WRITING FUNCTION
# =============================================================================
def write_output(collisions, file_name):
    with open(file_name, "w") as f:
        for collision in collisions:
            t, id1, id2 = collision
            f.write(f"{t:.1f} {id1} {id2}\n")

# =============================================================================
# ANIMATED 3D VISUALIZATION FUNCTION (WITH SPACE BACKGROUND)
# =============================================================================
def animate_simulation_3d(asteroids, dt, max_time):
    t_values = np.arange(0, max_time + dt, dt)
    trajectory_data = {a.id: {'x': [], 'y': []} for a in asteroids}
    
    for t in t_values:
        for a in asteroids:
            trajectory_data[a.id]['x'].append(a.x + a.vx * t)
            trajectory_data[a.id]['y'].append(a.y + a.vy * t)
    
    frames = []
    for i, t in enumerate(t_values):
        data = []
        for a in asteroids:
            data.append(go.Scatter3d(
                x=trajectory_data[a.id]['x'][:i+1],
                y=trajectory_data[a.id]['y'][:i+1],
                z=[0] * (i+1),
                mode='lines+markers',
                marker=dict(size=8),
                name=f'Asteroid {a.id}'
            ))
        frames.append(go.Frame(data=data, name=str(round(t, 1))))
    
    init_data = []
    for a in asteroids:
        init_data.append(go.Scatter3d(
            x=[a.x],
            y=[a.y],
            z=[0],
            mode='lines+markers',
            marker=dict(size=8),
            name=f'Asteroid {a.id}'
        ))
    
    fig = go.Figure(
        data=init_data,
        layout=go.Layout(
            title="Animated 3D Asteroid Simulation (2D Motion)",
            scene=dict(
                xaxis=dict(title="X Position (m)", backgroundcolor="black", color="white", gridcolor="gray"),
                yaxis=dict(title="Y Position (m)", backgroundcolor="black", color="white", gridcolor="gray"),
                zaxis=dict(title="Z Position (m)", backgroundcolor="black", color="white", gridcolor="gray"),
                bgcolor="black"
            ),
            paper_bgcolor='black',
            plot_bgcolor='black',
            font=dict(color='white'),
            updatemenus=[dict(
                type="buttons",
                showactive=False,
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None, {"frame": {"duration": 50, "redraw": True},
                                           "fromcurrent": True,
                                           "transition": {"duration": 0}}])]
            )]
        ),
        frames=frames
    )
    fig.show()

# =============================================================================
# MAIN FUNCTION
# =============================================================================
def main():
    input_file = "asteroids.txt"
    output_file = "collisions.txt"
    
    time_step = 0.1
    max_time = 10.0

    asteroids = parse_input(input_file)
    collisions = detect_collisions(asteroids, time_step, max_time)
    write_output(collisions, output_file)
    
    print(f"Simulation completed. Detected {len(collisions)} collision(s). Check '{output_file}' for results.")
    animate_simulation_3d(asteroids, time_step, max_time)

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'asteroids.txt'